needed to extract this kind of thing...

date --> dd/mm/yyyy=, hh:mm - username: message 

there could be a username or it coud be just a group
notification i need to handle that...


12/05/2024, 10:30 - Arijit: Hello there!
12/05/2024, 10:31 - Friend: Hey, what’s up?
12/05/2024, 10:32 - Arijit left the group

and it would only work when we pass any text to it otherwise not...


In [2]:
import re
import pandas as pd


# -------------------------
# Step 1: Regex pattern + splitting data
# -------------------------

In [3]:
def step1_split_dates_messages(data):
    """
    Returns: messages (list), dates (list), df (DataFrame with columns ['user_message','date_str'])
    """
    pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)
    df = pd.DataFrame({'user_message': messages, 'date': dates})
    return messages, dates, df

# -------------------------
# Step 2: Convert date column to datetime
# -------------------------

In [4]:
def step2_convert_dates(df):
    """
    Input: df with columns ['user_message','date'] where date is string timestamps
    Returns: df with 'date' converted to datetime (in-place copy returned)
    """
    df = df.copy() # why did this ?
    df['date'] = pd.to_datetime(df['date'], format=r'%d/%m/%Y, %H:%M - ')
    return df

# -------------------------
# Step 3: Prepare empty lists for users & messages
# -------------------------

In [5]:
def step3_prepare_lists():
    """
    Returns two empty lists: users, messages (so you can inspect before filling)
    """
    users = []
    messages = []
    return users, messages

# -------------------------
# Step 4: Split username and message for each entry (your original logic)
# -------------------------

In [6]:
def step4_split_user_message_lists(df):
    """
    Input: df with column 'user_message' (strings)
    Returns: users list, messages list (populated using your regex logic)
    NOTE: this uses your exact regex and join approach.
    """
    users = []
    messages = []
    for message in df['user_message']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:  # user name
            users.append(entry[1])
            messages.append(" ".join(entry[2:]))
        else:
            users.append('group_notification')
            messages.append(entry[0])
    return users, messages

<>:10: SyntaxWarning: invalid escape sequence '\w'
<>:10: SyntaxWarning: invalid escape sequence '\w'
/var/folders/vp/mhbkf2gs397d1pq5hhxflp6h0000gn/T/ipykernel_97596/88221220.py:10: SyntaxWarning: invalid escape sequence '\w'
  entry = re.split('([\w\W]+?):\s', message)


# -------------------------
# Step 5: Add user & message columns to DataFrame and drop raw column
# -------------------------

In [7]:
def step5_add_user_message_columns(df, users, messages):
    """
    Input:
      - df with 'user_message' and 'date'
      - users list and messages list (same length as df)
    Returns: df with new columns 'user' and 'message' and without 'user_message'
    """
    df = df.copy()
    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message'], inplace=True)
    return df

# -------------------------
# Step 6: Extract date/time components
# -------------------------

In [8]:
def step6_extract_datetime_parts(df):
    """
    Input: df with 'date' as datetime
    Adds: only_date, year, month_num, month, day, day_name, hour, minute
    Returns: df copy with those columns
    """
    df = df.copy()
    df['only_date'] = df['date'].dt.date
    df['year'] = df['date'].dt.year
    df['month_num'] = df['date'].dt.month
    df['month'] = df['date'].dt.month_name()
    df['day'] = df['date'].dt.day
    df['day_name'] = df['date'].dt.day_name()
    df['hour'] = df['date'].dt.hour
    df['minute'] = df['date'].dt.minute
    return df


# -------------------------
# Step 7: Create hourly period ranges (your exact code)
# -------------------------

In [9]:
def step7_create_period(df):
    """
    Input: df with 'hour' column
    Uses the same loop you wrote to build 'period' and returns df copy with 'period'
    """
    df = df.copy()
    period = []
    for hour in df['hour']:
        if hour == 23:
            period.append(str(hour) + "-" + str('0'))
        else:
            period.append(str(hour) + "-" + str(hour + 1))
    df['period'] = period
    return df

# -------------------------
# Full wrapper that runs your original steps in sequence (returns final df)
# -------------------------

In [10]:
def preprocess_split_steps(data):
    """
    Runs steps 1..7 in order but returns ALL intermediate results as a dict so you can inspect each stage.
    Returned dict keys:
      'messages', 'dates', 'df_step1',
      'df_step2', 'users', 'messages_list', 'df_step5', 'df_step6', 'df_step7'
    """
    # Step 1
    messages, dates, df_step1 = step1_split_dates_messages(data)

    # Step 2
    df_step2 = step2_convert_dates(df_step1)

    # Step 3
    # (we return empty lists too so user can inspect before/after)
    users_empty, messages_empty = step3_prepare_lists()

    # Step 4
    users, messages_list = step4_split_user_message_lists(df_step2)

    # Step 5
    df_step5 = step5_add_user_message_columns(df_step2, users, messages_list)

    # Step 6
    df_step6 = step6_extract_datetime_parts(df_step5)

    # Step 7
    df_step7 = step7_create_period(df_step6)

    return {
        'messages': messages,
        'dates': dates,
        'df_step1': df_step1,   # user_message (strings) + date (strings)
        'df_step2': df_step2,   # date as datetime
        'users_empty': users_empty,
        'messages_empty': messages_empty,
        'users': users,
        'messages_list': messages_list,
        'df_step5': df_step5,   # after adding user/message columns
        'df_step6': df_step6,   # after datetime parts
        'df_step7': df_step7    # final df with period
    }

In [11]:
for message in  df[message]:
    print(message)

NameError: name 'df' is not defined